In [ ]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from scipy.stats import entropy
import numpy as np
import gc
import shapely
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

bu = gpd.read_parquet('data/buurten_concatenated.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [ ]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

In [ ]:
bu.head(3)

In [ ]:
for idx, row in tqdm(bu.iterrows(), total=bu.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        bu.at[idx, f'L0_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L0_category.values)
        bu.at[idx, f'L1_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L1_category.values)
        
        for cat in L0_cats:
            bu.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            bu.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

bu.to_parquet('data/buurten_concatenated_sh.parquet') 